# Loadbalance Between Two OpenAI Accounts

Simple code example on implementing load balancing with Portkey.

### 1. Create **`two LLM objects`** with **`two different API keys`**

In [ ]:
# Install the Portkey SDK
!pip install portkey-ai

# Import necessary modules from Portkey
import portkey
from portkey import Config, LLMOptions

# Create two LLM objects with different account keys
llm_a = LLMOptions(provider = "openai", api_key = "OPENAI_ACCOUNT_1_KEY")
llm_b = LLMOptions(provider = "openai", api_key = "OPENAI_ACCOUNT_2_KEY")

### 2. Implement **`loadbalance`** mode

In [ ]:
# Construct the Porktey client and pass the LLM objects

portkey.config = Config(
    api_key = "PORTKEY_API_KEY", 
    mode = "loadbalance", 
    llms = [llm_a,llm_b]
)

### 3. Make the **`ChatComplete`** call

**Portkey SDK follows OpenAI SDK signature** - model params **including the model name**, can be passed at the completion call.

Just replce **`openai.ChatCompletion.create`** to **`portkey.ChatCompletion.create`** - **Everything else remains the same**

In [ ]:
response = portkey.ChatCompletions.create(
    model = "gpt-4-0613", 
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the meaning of life, universe and everything?"},
    ]
)

print(response)